## Two novel expvar

In [ ]:
# region

figure = Fig()
fig, gs = figure.draw(num=1, grid=(2, 2))
sessions = subjects.Tn().ratSday5

for sub, sess in enumerate(sessions):

    pre = sess.epochs.pre

    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2
    post1 = sess.epochs.post1
    post2 = sess.epochs.post2

    # --- break region into periods --------
    bin1 = sess.utils.getinterval(pre, 2)
    bin2 = sess.utils.getinterval(post2, 3)
    bins = bin1 + bin2
    sess.spikes.stability.firingRate(periods=bins)

    # ----- expvar maze1 -------
    control = pre
    template = maze1  # [post[0] + 4 * 3600, post[0] + 5 * 3600]
    match = [post1[0], post2[1]]  # post2  # [post[0] + 5 * 3600, post[1]]

    sess.replay.expvar.compute(
        template=template,
        match=match,
        control=control,
        slideby=300,
        cross_shanks=True,
    )
    print(sess.replay.expvar.npairs)

    axstate = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs[sub], hspace=0.2)

    ax1 = fig.add_subplot(axstate[1:])
    sess.replay.expvar.plot(ax=ax1, tstart=post1[0])
    ax1.set_xlim(left=0)
    ax1.tick_params(width=2)

    # ----- expvar maze2 -------
    control = pre
    template = maze2  # [post[0] + 4 * 3600, post[0] + 5 * 3600]
    match = post2  # [post[0] + 5 * 3600, post[1]]

    sess.replay.expvar.colors["ev"] = "#e75fdc"
    sess.replay.expvar.compute(
        template=template,
        match=match,
        control=control,
        slideby=300,
        cross_shanks=True,
    )
    print(sess.replay.expvar.npairs)

    sess.replay.expvar.plot(ax=ax1, tstart=post1[0])

    maze_time = (np.array(maze2) - post1[0]) / 3600
    ax1.axvspan(xmin=maze_time[0], xmax=maze_time[1], color="#f2a1a1", alpha=0.5)

ax1.legend(["rev_maze1", "maze1", "rev_maze2", "maze2"])
# figure.savefig("EV_sessions", __file__)

# endregion

## Reactivation of maze1 on maze2 (Two-novel) **excluding** pbe/ripple
Using traditional explained variance measure we found maze1 reactivates during maze2. However, is it also the case if you exclude pbe/replay periods on maze2 

In [2]:
import numpy as np
import subjects
import matplotlib.pyplot as plt
from plotUtil import Fig
import pandas as pd
import pingouin as pg

In [3]:
%matplotlib qt
figure = Fig()
fig,gs= figure.draw(num=1,grid=(2,2))
sessions = subjects.Tn().ratSday5
for sub,sess in enumerate(sessions):
    pbe = sess.pbe.events
    rpls = sess.ripple.events
    pyrid = sess.spikes.pyrid
    pre = sess.epochs.pre
    maze1 = sess.epochs.maze1
    maze2 = sess.epochs.maze2
    post1 = sess.epochs.post1
    post2 = sess.epochs.post2

    #--- ripples within maze-----
    rpls = rpls[(rpls.start>maze2[0]) & (rpls.start<maze2[1])].reset_index()

    # # --- break region into periods --------
    # bin1 = sess.utils.getinterval(pre, 2)
    # bin2 = sess.utils.getinterval(post2, 3)
    # bins = bin1 + bin2
    # sess.spikes.stability.firingRate(periods=bins)

    corr_pre,t_pre = sess.spikes.corr_pairwise(pyrid,pre,cross_shanks=True,window=900,slideby=300)

    corr_maze1 = sess.spikes.corr_pairwise(pyrid,maze1,cross_shanks=True)[0]

    inter_ripple_periods = pd.DataFrame({'start':rpls.end.values[:-1],'end':rpls.start.values[1:]}) 
    spks = sess.spikes.pyr
    spkcounts =[]
    for epoch in inter_ripple_periods.itertuples():
        bins = np.arange(epoch.start,epoch.end+0.25,0.25)
        spkcnt_ = np.asarray([np.histogram(_,bins=bins)[0] for _ in spks])
        # spkcnt_ = sess.spikes.get_spkcounts(cell_ids=pyrid,period=[epoch.start,epoch.end],binsize=0.250)
        spkcounts.append(spkcnt_)
    spkcounts = np.hstack(spkcounts)

    spkcounts = np.lib.stride_tricks.sliding_window_view(spkcounts,3600,axis=1)[:,::1200,:] 

    #--- cross shanks pairs--------
    shnkId = sess.spikes.get_shankID(pyrid)
    selected_pairs = np.nonzero(
                np.tril(shnkId.reshape(-1, 1) - shnkId.reshape(1, -1))
            )
    corr_maze2 = []
    for i in range(spkcounts.shape[1]):
        corr_maze2.append(np.corrcoef(spkcounts[:,i,:])[selected_pairs])
    corr_maze2 = np.asarray(corr_maze2).T

    template = corr_maze1
    corr = np.zeros((corr_maze2.shape[1],corr_pre.shape[1]))
    for w1 in range(corr_maze2.shape[1]):
        match_corr = corr_maze2[:,w1]
        for w2 in range(corr_pre.shape[1]):
            control_corr = corr_pre[:,w2]
            df = pd.DataFrame({'x':template,'y':match_corr,'control':control_corr})
            corr[w1,w2] = pg.partial_corr(data=df,x='x',y='y',covar='control')['r2']




# ax1.legend(["rev_maze1", "maze1", "rev_maze2", "maze2"])


In [4]:
ev = np.mean(corr,axis=1)
t_maze2 = np.arange(0,8*5,5)
plt.plot(t_maze2,ev,'k')
plt.ylabel('Explained variance')
plt.xlabel('Maze2 Bins (minutes)')
plt.title('Reactivation of Maze1 on Maze2 after excluding \n ripple periods')

Text(0.5, 1.0, 'Reactivation of Maze1 on Maze2 after excluding \n ripple periods')